# Master Metrcis for L2 Retrospecive Grants

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ramshreyas/retrospective_grants_metrics/blob/main/master_metrics.ipynb)

---

### Setup & Imports

In [1]:
import pandas as pd

Load data

In [2]:
projects_df = pd.read_csv('data/L2 Community Grants Retrospective - L2 Retro Opportunity.csv')
projects_df.head()

,Opportunity Name,Account Name,Website,Github,Was Project Completed?,Can we consider it a sucess ?,Is it still a public good?,Does the Project need additional funding?,Have they raised funding?,What is the team doing now?,...,Proactive Community Grants Round,Category,Cost Center,Opportunity ID,Country,Countries of Team,Contract Number,Next Deliverable Date,Last Stage Change Date,Stage Duration
0,Starksheet,Starksheet,https://starksheet.notion.site/Starksheet-bfb5...,https://github.com/the-candy-shop/starksheet-m...,NaN,NaN,NaN,NaN,NaN,NaN,...,Layer 2 Community Grants 2022,UX/UI,Privacy and Scaling Explorations,0065J00000Ck1lV,FR,NaN,896,NaN,8/10/2023,245
1,MyFirstLayer2,LXDAO,forum.lxdao.io/t/my-first-layer2/240/16?u=bruc...,https://github.com/lxdao-official/myfirstlayer...,NaN,NaN,NaN,NaN,NaN,NaN,...,Layer 2 Community Grants 2022,Community and education,Privacy and Scaling Explorations,0065J00000Ck1la,CN,"China, New Zealand, Japan, U.K., U.S.",890,NaN,7/7/2023,279
2,Typescript ERC-4337 Bundler,EntryPoint,hackmd.io/@kristofgazso/rJco0WrIi,https://github.com/pimlicolabs/alto,NaN,NaN,NaN,NaN,NaN,NaN,...,Layer 2 Community Grants 2022,UX/UI,Privacy and Scaling Explorations,0065J00000Ck1lf,GB,NaN,897,NaN,9/26/2023,198
3,ERC-4337 Public Infrastructure,Candide,github.com/candidelabs,github.com/candidelabs,Yes,NaN,NaN,NaN,NaN,NaN,...,Layer 2 Community Grants 2022,Cybersecurity,Privacy and Scaling Explorations,0065J00000Ck1lk,FR,"Kuweit, Egypt",884,NaN,8/17/2023,238
4,Rollup Security Framework,"Quantstamp, Inc.",drive.google.com/file/d/1bK0bkY47tj8pY192DHME3...,https://github.com/quantstamp/l2-security-fram...,NaN,NaN,NaN,NaN,NaN,NaN,...,Layer 2 Community Grants 2022,Cybersecurity,Privacy and Scaling Explorations,0065J00000Ck1ll,CA,Canada,894,NaN,8/24/2023,231


Initialize metrics sheet

In [3]:
metrics_df = projects_df[['Opportunity Name', 'Account Name', 'Github']]
metrics_df['Start Date'] = None
metrics_df.head()

/tmp/ipykernel_471270/2244965679.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics_df['Start Date'] = None


,Opportunity Name,Account Name,Github,Start Date
0,Starksheet,Starksheet,https://github.com/the-candy-shop/starksheet-m...,None
1,MyFirstLayer2,LXDAO,https://github.com/lxdao-official/myfirstlayer...,None
2,Typescript ERC-4337 Bundler,EntryPoint,https://github.com/pimlicolabs/alto,None
3,ERC-4337 Public Infrastructure,Candide,github.com/candidelabs,None
4,Rollup Security Framework,"Quantstamp, Inc.",https://github.com/quantstamp/l2-security-fram...,None


---

### Stars